In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian
import var_opt

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

In [2]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

In [3]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk}           

In [4]:
#pr = lih.pauli_rep  #OK with iterative updates
#pr = beh2.pauli_rep #OK with iterative updates

In [5]:
ham = lih_small
num_qubits = ham.pauli_rep.num_qubits
energy, state = ham.pauli_rep.ground()
β_optimal_diagonal = ham.pauli_rep.local_dists_optimal('diagonal')

objective = var_opt.objective_diagonal(ham.pauli_rep.dic_tf, β_optimal_diagonal) # the cost function's value

print("beta_optimal_diagonal:")
print(β_optimal_diagonal)
print('===')
print("objective:", objective)

beta_optimal_diagonal:
{0: [0.2627877783884178, 0.25172728999060096, 0.4854849316209812], 1: [0.2731407677187616, 0.2708496665000182, 0.45600956578122026], 2: [0.2627877734835786, 0.25172728735894984, 0.4854849391574715], 3: [0.2731407699348335, 0.2708496658369026, 0.45600956422826394]}
===
objective: 2.6472360929681003


In [6]:
def prod_inverse(Q, β):
    # prod_{j in supp(Q)} beta_{j, Q_j}^{-1}
    pauli_to_index = {'X': 0, 'Y': 1, 'Z': 2}
    prod = 1.0
    for i in range(len(Q)):
        if Q[i] != 'I':
            qubit = (len(Q)-1)-i  # qiskit ordering: if i=0 then need to look up beta[n]
            index = pauli_to_index[Q[i]]
            b = β[qubit][index]
            prod *= b
    return prod**(-1)

def lagrange_restriction_numerator(i, p, dic_tf, β):
    assert p in ['X', 'Y', 'Z']
    tally = 0.0
    for Q in dic_tf:
        if Q[(len(Q)-1)-i] == p: # qiskit ordering
            tally += (dic_tf[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction_denominator(i, dic_tf, β):
    tally = 0.0
    for Q in dic_tf:
        if Q[(len(Q)-1)-i] != "I": # qiskit ordering
            tally += (dic_tf[Q]**2) * prod_inverse(Q, β)
    return tally

def lagrange_restriction(i, p, dic_tf, β):
    numerator = lagrange_restriction_numerator(i, p, dic_tf, β)
    denominator = lagrange_restriction_denominator(i, dic_tf, β)
    return numerator / denominator

In [7]:
#THIS IS  WHERE WE USE ITERATIVE UPDATES TO FIND THE BEST BETAS

import numpy as np

def distance(β_1, β_2):
    two_norm_squared = 0.0
    for qubit in β_1.keys(): # qubit is qubit number (in qiskit ordering)
        two_norm_squared_qubit = np.sum( (np.array(β_1[qubit]) - np.array(β_2[qubit]))**2 )
        two_norm_squared += two_norm_squared_qubit
    return np.sqrt(two_norm_squared)

def update_betas(dic_tf, num_qubits, β_old = None, weight=0.1):
    """
        iteratively update betas with new values by weight 
    """
    if β_old is None: #initialize with random uniform
        β_old = {}
        for qubit in range(num_qubits):
            β_old[qubit] = [ 1./3 for _ in range(3) ]
    β_new = {}
    for qubit in range(num_qubits):
        β_new[qubit] = []
        for index, pauli in enumerate(("X", "Y", "Z")):
            lagrange_rest = lagrange_restriction(qubit, pauli, dic_tf, β_old)
            update = (1.- weight) * β_old[qubit][index] + weight * lagrange_rest
            β_new[qubit].append(update)
    return β_new, distance(β_new, β_old)

In [8]:
data = ( "lih_small", lih_small)

In [9]:
def test_iterative_updates(tol=1.0e-5):
    for name, ham in hamiltonians.items():
        
        print("Molecule=", name, "------------------")
        num_qubits = ham.pauli_rep.num_qubits
        energy, state = ham.pauli_rep.ground()
        β_optimal_diagonal = ham.pauli_rep.local_dists_optimal('diagonal')
        objective = var_opt.objective_diagonal(ham.pauli_rep.dic_tf, β_optimal_diagonal) # the cost function's value
         
        # THIS IS THE EXPERIMENT WITH ITERATIVE UPDATES TO FIND BETAS 
        tol = 1.0e-5
        iter = 0
        β_old = None
        while True and iter < 1000:
            β_new, error = update_betas(ham.pauli_rep.dic_tf, num_qubits, β_old)
            β_old = β_new
            print("\t", iter, error)
            iter += 1
            if error < tol:
                print("MOLECULE", name, "CONVERGE with error = ", error)
                break

        print("Distance from true values:", distance(β_new, β_optimal_diagonal)) 

In [10]:
test_iterative_updates()

Molecule= lih_small ------------------
	 0 0.11261586135042559
	 1 0.07825628474170102
	 2 0.05202240430061836
	 3 0.033880031598654124
	 4 0.021925624185318147
	 5 0.014210407724462013
	 6 0.009276649946109662
	 7 0.006139246356234342
	 8 0.004154674175809936
	 9 0.002905510613624632
	 10 0.002119806498677156
	 11 0.0016198954727863284
	 12 0.0012915104421728637
	 13 0.0010641692440765077
	 14 0.000896725360677269
	 15 0.0007662223857614373
	 16 0.0006600844415156557
	 17 0.0005713071286726417
	 18 0.0004957830223325781
	 19 0.00043090791830143344
	 20 0.0003748793822691835
	 21 0.00032634781583358477
	 22 0.00028424134968488756
	 23 0.00024767520732789155
	 24 0.00021590205284284985
	 25 0.00018828245762566505
	 26 0.00016426553446891726
	 27 0.00014337498727634579
	 28 0.0001251982845021868
	 29 0.00010937782368206882
	 30 9.56035007943388e-05
	 31 8.360635727727898e-05
	 32 7.315310258536152e-05
	 33 6.40413728700628e-05
	 34 5.609562006280731e-05
	 35 4.916354569557362e-05
	 36 4.

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


	 0 0.08601299568057812
	 1 0.06785095529261422
	 2 0.05336477140346753
	 3 0.041895919174394614
	 4 0.0330127509032797
	 5 0.026238967828564923
	 6 0.021095595700498796
	 7 0.01716571559090108
	 8 0.014122504144670376
	 9 0.011726352716024036
	 10 0.00980791700876857
	 11 0.008249025482684047
	 12 0.006966774207784095
	 13 0.0059019694877530116
	 14 0.0050113062598735195
	 15 0.004262255039195648
	 16 0.0036297564160765895
	 17 0.003094073164009261
	 18 0.0026393714790396384
	 19 0.002252761189902437
	 20 0.0019236279237627076
	 21 0.0016431544228918255
	 22 0.0014039673545136815
	 23 0.001199869673944053
	 24 0.0010256330158221704
	 25 0.000876833432031736
	 26 0.0007497192973433829
	 27 0.0006411036876881489
	 28 0.0005482757869589462
	 29 0.0004689273657569065
	 30 0.00040109138261634736
	 31 0.0003430904575660451
	 32 0.0002934934660613148
	 33 0.00025107886519071997
	 34 0.000214803636187312
	 35 0.00018377693523580067
	 36 0.0001572377065691678
	 37 0.00013453564015869415
	 38 0

	 0 0.13703045680358492
	 1 0.12176919707590238
	 2 0.10797307480423644
	 3 0.09555901696143272
	 4 0.08458538740888115
	 5 0.07498742012555955
	 6 0.06661545075244509
	 7 0.05929888571566724
	 8 0.05288098612646971
	 9 0.04722918313829185
	 10 0.04223417204372722
	 11 0.037805965254411135
	 12 0.033870021588895585
	 13 0.03036411339351111
	 14 0.02723585727427325
	 15 0.02444075636116801
	 16 0.021940654216782724
	 17 0.019702532072742186
	 18 0.0176975872277526
	 19 0.01590053122900592
	 20 0.014289051847296017
	 21 0.012843392946229075
	 22 0.011546017917217837
	 23 0.01038133272695206
	 24 0.009335452618424101
	 25 0.008396002060559994
	 26 0.007551941124873299
	 27 0.006793413663938535
	 28 0.0061116139749504686
	 29 0.005498669411208812
	 30 0.004947536886275055
	 31 0.004451911536716454
	 32 0.004006146045217135
	 33 0.0036051793152088356
	 34 0.003244473349665867
	 35 0.0029199573286846346
	 36 0.002627978006511444
	 37 0.0023652556604668684
	 38 0.0021288449228099716
	 39 0.00